In [ ]:
import jtrace
import os
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
HSC_fn = os.path.join(jtrace.datadir, "hsc", "HSC.yaml")
HSC_telescope = jtrace.Telescope.makeFromYAML(HSC_fn)

In [ ]:
LSST_fn = os.path.join(jtrace.datadir, "lsst", "LSST_r.yaml")
LSST_telescope = jtrace.Telescope.makeFromYAML(LSST_fn)

In [ ]:
DECam_fn = os.path.join(jtrace.datadir, "decam", "DECam.yaml")
DECam_telescope = jtrace.Telescope.makeFromYAML(DECam_fn)

In [ ]:
def dthdr(telescope, th, pixelSize=15):
    rays0 = jtrace.parallelRays(10, 4.1, 0.0, theta_x=th)
    rays1 = jtrace.parallelRays(10, 4.1, 0.0, theta_x=th+1e-5)
    rays2 = jtrace.parallelRays(10, 4.1, 0.0, theta_x=th, theta_y=1e-5)

    traced_rays0 = telescope.trace(rays0)
    traced_rays1 = telescope.trace(rays1)
    traced_rays2 = telescope.trace(rays2)

    traced_rays0 = jtrace.RayVector([r for r in traced_rays0 if not r.isVignetted])
    traced_rays1 = jtrace.RayVector([r for r in traced_rays1 if not r.isVignetted])
    traced_rays2 = jtrace.RayVector([r for r in traced_rays2 if not r.isVignetted])

    drx_dthx = (np.mean(traced_rays1.x) - np.mean(traced_rays0.x))/1e-5 # m / radian
    drx_dthx *= 1e6 # micron / radian
    drx_dthx /= 206265 # micron / arcsec
    drx_dthx /= pixelSize # pixels / arcsec

    drx_dthy = (np.mean(traced_rays2.x) - np.mean(traced_rays0.x))/1e-5 # m / radian
    drx_dthy *= 1e6 # micron / radian
    drx_dthy /= 206265 # micron / arcsec
    drx_dthy /= pixelSize # pixels / arcsec

    dry_dthx = (np.mean(traced_rays1.y) - np.mean(traced_rays0.y))/1e-5 # m / radian
    dry_dthx *= 1e6 # micron / radian
    dry_dthx /= 206265 # micron / arcsec
    dry_dthx /= pixelSize # pixels / arcsec

    dry_dthy = (np.mean(traced_rays2.y) - np.mean(traced_rays0.y))/1e-5 # m / radian
    dry_dthy *= 1e6 # micron / radian
    dry_dthy /= 206265 # micron / arcsec
    dry_dthy /= pixelSize # pixels / arcsec
    
    dr_dth = np.array([[drx_dthx, dry_dthx], [drx_dthy, dry_dthy]])
    dth_dr = np.linalg.inv(dr_dth)

    return dth_dr

In [ ]:
dthx_drx = []
dthy_dry = []
thetas = np.linspace(0.0, 0.75, 10)
for th in thetas:
    dth_dr = dthdr(HSC_telescope, th*3600/206265)
    dthx_drx.append(abs(dth_dr[0,0]))
    dthy_dry.append(abs(dth_dr[1,1]))
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
ax1.plot(thetas, dthx_drx, c='b', label='radial')
ax1.plot(thetas, dthy_dry, c='r', label='tangential')
ax1.set_xlabel('radius (deg)')
ax1.set_ylabel('pixel size (arcsec)')
ax1.legend(loc='lower left')
ax1.set_title("HSC")

ax2.plot(thetas, np.array(dthx_drx)/np.array(dthy_dry))
ax2.set_xlabel('radius (deg)')
ax2.set_ylabel('b/a')
ax2.set_title("HSC")

fig.tight_layout()
fig.show()

In [ ]:
dthx_drx = []
dthy_dry = []
thetas = np.linspace(0.0, 1.75, 10)
for th in thetas:
    dth_dr = dthdr(LSST_telescope, th*3600/206265, pixelSize=10)
    dthx_drx.append(abs(dth_dr[0,0]))
    dthy_dry.append(abs(dth_dr[1,1]))
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
ax1.plot(thetas, dthx_drx, c='b', label='radial')
ax1.plot(thetas, dthy_dry, c='r', label='tangential')
ax1.set_xlabel('radius (deg)')
ax1.set_ylabel('pixel size (arcsec)')
ax1.legend(loc='lower left')
ax1.set_title("LSST")

ax2.plot(thetas, np.array(dthx_drx)/np.array(dthy_dry))
ax2.set_xlabel('radius (deg)')
ax2.set_ylabel('b/a')
ax2.set_title("LSST")

fig.tight_layout()
fig.show()

In [ ]:
dthx_drx = []
dthy_dry = []
thetas = np.linspace(0.0, 1.1, 10)
for th in thetas:
    dth_dr = dthdr(DECam_telescope, th*3600/206265)
    dthx_drx.append(abs(dth_dr[0,0]))
    dthy_dry.append(abs(dth_dr[1,1]))
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
ax1.plot(thetas, dthx_drx, c='b', label='radial')
ax1.plot(thetas, dthy_dry, c='r', label='tangential')
ax1.set_xlabel('radius (deg)')
ax1.set_ylabel('pixel size (arcsec)')
ax1.legend(loc='lower left')
ax1.set_title("DECam")

ax2.plot(thetas, np.array(dthx_drx)/np.array(dthy_dry))
ax2.set_xlabel('radius (deg)')
ax2.set_ylabel('b/a')
ax2.set_title("DECam")

fig.tight_layout()
fig.show()